#Data Mining - Lab 03
#Ho va ten: Nguyen Duc Vu Duc
#MSSV: 18110004

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Import dataset
path='/content/diabetes_null.csv'
df=pd.read_csv(path)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,5,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,4.0,35.0,168.0,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   763 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   Insulin                   394 non-null    float64
 5   BMI                       757 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 54.1 KB


Xuất các đặc trưng và kèm theo số lượng dòng NULL/NAN của từng cột đó

In [5]:
for column in df.columns:
  print('The number of NaN values in {} columns: {}'.format(column,df[column].isna().sum()))

The number of NaN values in Pregnancies columns: 0
The number of NaN values in Glucose columns: 5
The number of NaN values in BloodPressure columns: 35
The number of NaN values in SkinThickness columns: 227
The number of NaN values in Insulin columns: 374
The number of NaN values in BMI columns: 11
The number of NaN values in DiabetesPedigreeFunction columns: 0
The number of NaN values in Age columns: 0
The number of NaN values in Outcome columns: 0


Đặc trưng Age là đặc trưng không có giá trị NULL/NAN nên ta chọn đặc trưng Age là đặc trưng chính. Thực hiện tạo ra 2 bộ data : 

Data 1 : gom nhóm Age theo hình thành các nhóm sau đó nếu giá trị NULL/NAN của các đặc trưng khác mà nằm trong nhóm nào thì thay bằng mean của nhóm đó 

In [6]:
df_1=df.copy()
df_1['Age group']=np.floor(df_1.loc[:,'Age']/10)
df_1=df_1.fillna(df_1.mean())
df_1=df_1.drop(['Age'],axis=1)
df_1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Outcome,Age group
0,6,148.0,72.0,35.000000,105.659898,33.6,0.627,1,0.0
1,1,85.0,66.0,29.000000,105.659898,26.6,0.351,0,3.0
2,8,183.0,64.0,25.876155,105.659898,23.3,0.672,1,3.0
3,1,89.0,66.0,23.000000,94.000000,28.1,0.167,0,2.0
4,0,137.0,4.0,35.000000,168.000000,43.1,2.288,1,3.0


Data 2 : góm nhóm thành 5 nhóm đều nhau sau đó nếu giá trị NULL/NAN của các đặc trưng khác mà nằm trong nhóm nào thì thay bằng mean của nhóm đó 


In [7]:
from sklearn import preprocessing
df_2=df.copy()
df_2['Age group'] = pd.cut(df_2['Age'], bins=5)
df_2=df_2.fillna(df_2.mean())
df_2=df_2.drop(['Age'],axis=1)
le = preprocessing.LabelEncoder()
le.fit(df_2['Age group'])
df_2['Age group']=le.transform(df_2['Age group'])
df_2=pd.DataFrame(df_2)
df_2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Outcome,Age group
0,6,148.0,72.0,35.000000,105.659898,33.6,0.627,1,0
1,1,85.0,66.0,29.000000,105.659898,26.6,0.351,0,1
2,8,183.0,64.0,25.876155,105.659898,23.3,0.672,1,1
3,1,89.0,66.0,23.000000,94.000000,28.1,0.167,0,1
4,0,137.0,4.0,35.000000,168.000000,43.1,2.288,1,1


Chạy các thuật toán Classification KNN, Decision Tree, Random Forest, Naïve Bayes

In [8]:
#Import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [9]:
#Preprocessing dataset
data_1=pd.concat([df_1.iloc[:,:7],df_1.iloc[:,8]],axis=1)
data_1_label=df_1.iloc[:,7]

data_2=pd.concat([df_2.iloc[:,:7],df_2.iloc[:,8]],axis=1)
data_2_label=df_2.iloc[:,7]

In [10]:
#Train test split 2 dataset 
X_train_1,X_test_1, y_train_1,y_test_1=train_test_split(data_1,data_1_label,test_size=0.2,random_state=42)
X_train_2,X_test_2, y_train_2,y_test_2=train_test_split(data_2,data_2_label,test_size=0.2,random_state=42)

In [11]:
#K Nearest Neighbors Classifier
neigh_1 = KNeighborsClassifier(n_neighbors=10)
neigh_1.fit(X_train_1, y_train_1)
y_1_knn_pred= neigh_1.predict(X_test_1)
print('Accuracy of KNN on dataset 1: ',np.sum(y_1_knn_pred == y_test_1)/len(y_test_1))

neigh_2 = KNeighborsClassifier(n_neighbors=5)
neigh_2.fit(X_train_2, y_train_2)
y_2_knn_pred= neigh_2.predict(X_test_2)
print('Accuracy of KNN on dataset 2: ',np.sum(y_2_knn_pred == y_test_2)/len(y_test_2))

#Decision Tree
clf_1 = DecisionTreeClassifier()
clf_1.fit(X_train_1, y_train_1)
y_1_dt_pred=clf_1.predict(X_test_1)
print('Accuracy of Decision Tree on dataset 1: ',np.sum(y_1_dt_pred == y_test_1)/len(y_test_1))

clf_2 = DecisionTreeClassifier()
clf_2.fit(X_train_2, y_train_2)
y_2_dt_pred=clf_2.predict(X_test_2)
print('Accuracy of Decision Tree on dataset 2: ',np.sum(y_2_dt_pred == y_test_2)/len(y_test_2))

#Random Forest
rf_1 = RandomForestClassifier(max_depth=5, random_state=0)
rf_1.fit(X_train_1, y_train_1)
y_1_rf_pred=rf_1.predict(X_test_1)
print('Accuracy of Random Forest on dataset 1: ',np.sum(y_1_rf_pred == y_test_1)/len(y_test_1))

rf_2 = RandomForestClassifier(max_depth=5, random_state=0)
rf_2.fit(X_train_2, y_train_2)
y_2_rf_pred=rf_2.predict(X_test_2)
print('Accuracy of Random Forest on dataset 2: ',np.sum(y_2_rf_pred == y_test_2)/len(y_test_2))

#Naive Bayes
nb_1=GaussianNB()
nb_1.fit(X_train_1,y_train_1)
y_1_nb_pred=nb_1.predict(X_test_1)
print('Accuracy of Naive Bayes on dataset 1: ',np.sum(y_1_nb_pred == y_test_1)/len(y_test_1))

nb_2=GaussianNB()
nb_2.fit(X_train_2,y_train_2)
y_2_nb_pred=nb_2.predict(X_test_2)
print('Accuracy of Naive Bayes on dataset 2: ',np.sum(y_2_nb_pred == y_test_2)/len(y_test_2))

Accuracy of KNN on dataset 1:  0.7142857142857143
Accuracy of KNN on dataset 2:  0.6818181818181818
Accuracy of Decision Tree on dataset 1:  0.6818181818181818
Accuracy of Decision Tree on dataset 2:  0.6818181818181818
Accuracy of Random Forest on dataset 1:  0.7467532467532467
Accuracy of Random Forest on dataset 2:  0.7532467532467533
Accuracy of Naive Bayes on dataset 1:  0.7402597402597403
Accuracy of Naive Bayes on dataset 2:  0.7532467532467533


Hãy tự phân tích rồi nghĩ ra cách phân nhóm hay thay thế sao cho hiệu quả để có độ phân loại classification tốt hơn 

In [12]:
df_3=df.copy()
df_3['Age group'] = pd.qcut(df_3['Age'], q=5)
df_3=df_3.fillna(df_3.median())
df_3=df_3.drop(['Age'],axis=1)
le = preprocessing.LabelEncoder()
le.fit(df_3['Age group'])
df_3['Age group']=le.transform(df_3['Age group'])
df_3=pd.DataFrame(df_3)
df_3.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Outcome,Age group
0,6,148.0,72.0,35.0,71.0,33.6,0.627,1,0
1,1,85.0,66.0,29.0,71.0,26.6,0.351,0,2
2,8,183.0,64.0,27.0,71.0,23.3,0.672,1,3
3,1,89.0,66.0,23.0,94.0,28.1,0.167,0,0
4,0,137.0,4.0,35.0,168.0,43.1,2.288,1,3


In [13]:
data_3=pd.concat([df_3.iloc[:,:7],df_3.iloc[:,8]],axis=1)
data_3_label=df_3.iloc[:,7]
X_train_3,X_test_3, y_train_3,y_test_3=train_test_split(data_3,data_3_label,test_size=0.2,random_state=42)

In [14]:
neigh_3 = KNeighborsClassifier(n_neighbors=5)
neigh_3.fit(X_train_3, y_train_3)
y_3_knn_pred= neigh_3.predict(X_test_3)
print('Accuracy of KNN on dataset 3: ',np.sum(y_3_knn_pred == y_test_3)/len(y_test_3))

clf_3 = DecisionTreeClassifier(max_depth=3,random_state=0)
clf_3.fit(X_train_3, y_train_3)
y_3_dt_pred=clf_3.predict(X_test_3)
print('Accuracy of Decision Tree on dataset 3: ',np.sum(y_3_dt_pred == y_test_3)/len(y_test_3))

rf_3 = RandomForestClassifier(max_depth=4, random_state=0)
rf_3.fit(X_train_1, y_train_1)
y_3_rf_pred=rf_3.predict(X_test_3)
print('Accuracy of Random Forest on dataset 3: ',np.sum(y_3_rf_pred == y_test_3)/len(y_test_3))

nb_3=GaussianNB()
nb_3.fit(X_train_3,y_train_3)
y_3_nb_pred=nb_3.predict(X_test_3)
print('Accuracy of Naive Bayes on dataset 3: ',np.sum(y_3_nb_pred == y_test_3)/len(y_test_3))

Accuracy of KNN on dataset 3:  0.6818181818181818
Accuracy of Decision Tree on dataset 3:  0.7337662337662337
Accuracy of Random Forest on dataset 3:  0.7727272727272727
Accuracy of Naive Bayes on dataset 3:  0.7597402597402597


Với cách chọn ở df_3 thì ta có độ chính xác của dataset trên 3 thuật toán Decision Tree, Random Forest, Naive Bayes cao hơn so với ở df_1 và df_2